In [2]:
import numpy as np
import pandas as pd 
import tensorflow as tf

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [3]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive 
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [5]:
!pip install transformers

     |████████████████████████████████| 2.5MB 5.3MB/s 
     |████████████████████████████████| 901kB 38.5MB/s 
     |████████████████████████████████| 3.3MB 39.7MB/s 


# Importing BERT pre-trained model

In [6]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

# Data Pre-processing

In [7]:
nltk.download("stopwords")
nltk.download('punkt')
stop_words= set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
def remove_stopwods(df,label):
  j=0
  for i in df[label]:
    text_tokens=word_tokenize(i)
    filtered_words=[word for word in text_tokens if word not in stop_words]
    df[label][j]=filtered_words
    j=j+1

In [9]:
data = pd.read_csv("/content/gdrive/My Drive/jio_data/jio_news_finaldata.csv")

In [10]:
data=data[["parsed_content"]]

In [11]:
data

,parsed_content
0,Three months after air transport data giant S...
1,A man wearing a protective face shield cleans...
2,A broker reacts while trading at his computer...
3,3D printed Facebook and Twitter logos are pla...
4,New updates are being added at the bottom of ...
...,...
759,"Juhi Chawla gripped by, well, Darr! One of the..."
760,"May 18, 2021 The speculation linking Amazon t..."
761,Jump to navigation Speak Now We are just a w...
762,The trial delivered a peak speed of over 1Gbp...


In [12]:
data['parsed_content']= data['parsed_content'].astype("str")

In [13]:
data['parsed_content'] = data['parsed_content'].apply(lambda x: x.lower())
data['parsed_content'] = data['parsed_content'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [14]:
remove_stopwods(data,"parsed_content")

In [15]:
def convertList(list1):  
    str = ''  # initializing the empty string  
    for i in list1: #Iterating and adding the list element to the str variable  
      str =str+ " "+i
    return str

In [16]:
k=0
while(k<len(data['parsed_content'])):
  data['parsed_content'][k]= convertList(data['parsed_content'][k])
  k=k+1

In [17]:
data['parsed_content'][0]

' three months air transport data giant sita reported data breach still learning damage air india said week personal data 45 million passengers compromised following incident sita indian flag carrier airlines data processor stolen information included passengers names credit card details date birth contact information passport information ticket information star alliance air india frequent flyer data air india said statement pdf cvvcvc data credit cards held sita said air india urged passengers change passwords wherever applicable ensure safety personal data attack compromised data passengers registered indian airline past decade august 26 2011 february 3 2021 air india said statement revelation comes months sita said suffered data breach involved passenger data time sita said notified several airlines malaysia airlines finnair singapore airlines jeju air cathay pacific air new zealand lufthansa breach geneva switzerlandheadquartered firm said serve 90 worlds airlines declined reveal s

# Prediction on jio news data

In [18]:
templabel_BERT=[]
tempscores_BERT=[]
sentiment_statement=[]
for i in data['parsed_content']:
  templabel_BERT.append(classifier(i[:1000])[0]['label'])
  tempscores_BERT.append(classifier(i[:1000])[0]['score'])
  sentiment_statement.append(i)

Here, we took only first thousand characters of news only in prediction because gist of news is mostly in the top 2-3 lines and BERT has limit of input to 512 words only

creating sentiment Label and score column

In [19]:
data["sentiment_label_bert"]= templabel_BERT
data["Sentiment_score"]= tempscores_BERT

In [20]:
data

,parsed_content,sentiment_label_bert,Sentiment_score
0,three months air transport data giant sita re...,NEGATIVE,0.995285
1,man wearing protective face shield cleans bha...,NEGATIVE,0.998522
2,broker reacts trading computer terminal stock...,NEGATIVE,0.967610
3,3d printed facebook twitter logos placed comp...,NEGATIVE,0.996269
4,new updates added bottom story original story...,NEGATIVE,0.624101
...,...,...,...
759,juhi chawla gripped well darr one popular mov...,NEGATIVE,0.983867
760,may 18 2021 speculation linking amazon major ...,NEGATIVE,0.877735
761,jump navigation speak week away 44th reliance...,NEGATIVE,0.991221
762,trial delivered peak speed 1gbps according re...,NEGATIVE,0.984361


In [23]:
data["sentiment_label_bert"][:300]

0      NEGATIVE
1      NEGATIVE
2      NEGATIVE
3      NEGATIVE
4      NEGATIVE
         ...   
295    POSITIVE
296    NEGATIVE
297    NEGATIVE
298    NEGATIVE
299    POSITIVE
Name: sentiment_label_bert, Length: 300, dtype: object